In [1]:
# import modules
import pandas as pd
import numpy as np
import spotipy

# full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas format
pd.set_option('display.float_format', '{:_.0f}'.format)
# NOTE: underscore separaters ('_') are better than commas (',') because 
# numbers with underscores work in Python without any extra effort.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# used for AUTHORIZATION WORKFLOW (abandoned)
# CONFIDENTIAL.py included in .gitignore, contains private API key
# use link above to get a client_id and client_secret from developer.spotify.com
from CONFIDENTIAL import client_id, client_secret, redirect_uri

### The Billboard 100

In [2]:
# Billboard Top 100 Historical Data
# link:  https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs
# via:  https://toolbox.google.com/datasetsearch
url_billboard = r'D:\RYERSON\820\Datasets\Billboard The Hot 100 Songs\charts.csv'

df_billboard = pd.read_csv(url_billboard)
df_billboard['date'] = pd.to_datetime(df_billboard['date'])

### Unique Songs from The Billboard 100 Dataset

In [3]:
# just the songs on the billboard 100, once per song
df_billboard_songs = df_billboard[['song', 'artist']].drop_duplicates().sort_values(['artist', 'song']).reset_index(drop=True)

# add a blank id column
df_billboard_songs['id'] = ''

df_billboard_songs.shape

(29681, 3)

### 1.2M Songs with Metadata (csv)
https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs

In [5]:
# there are ~30k songs on the Billboard 100 list
# let's see how many are here:
# https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs
# via:  https://www.kaggle.com/datasets/
# Spotify 1.2M+ Songs
url_1M_songs = r'D:\RYERSON\820\Datasets\Spotify 1.2M+ Songs\tracks_features.csv'

# create the dataframe with the large number of songs metadata
df_1M_songs = pd.read_csv(url_1M_songs)

# make a list of song ids from the 1M dataset
metadata_ids = list(df_1M_songs.id)

### Get Spotify IDs using API
(to join with large datasets)

https://developer.spotify.com/terms/

##### Useful Spotify API Features

https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features

https://developer.spotify.com/documentation/web-api/reference/#/operations/get-recommendations

##### Other Links

my app:
https://developer.spotify.com/dashboard/applications/cd5ce2cb690543ff9967e817d4665543

a tutorial: 
https://www.youtube.com/watch?v=cU8YH2rhN6A&ab_channel=Elbert

In [ ]:
# # AUTHORIZATION WORKFLOW 
# # (doens't work anymore, I think I was throttled, but can't get any error messages)

# # https://developer.spotify.com/documentation/general/guides/authorization/scopes/
# # i think this is the right scope, but nothing works when I connect to my app
# scope = 'user-read-private'

# oauth = spotipy.SpotifyOAuth(
#     client_id=client_id,
#     client_secret=client_secret,
#     redirect_uri=redirect_uri,
#     scope=scope
# )

# cached_token = oauth.get_cached_token()

# # create a spotify object
# spotify = spotipy.Spotify(auth=cached_token['access_token'])

### TEMPORARY TOKEN WORKFLOW

##### get a temporary authorization token from: https://developer.spotify.com/console/get-search-item

In [90]:
# input the temporary token
TEMP_TOKEN = input('Enter token: ')

Enter token: BQCGN3A6E1VE_V1reluA4eouLmf8gP0h0SOufnJP4eOJdPNxANR45RBLeJNoz8Nur6LxL4Fct7e0eSkdhKxWmBbK12S5RWL2Katsejj5cW3rQJ-4H8-3HS0zoiF71rMjkxRzqaWZmeaeuuV3XdhxGjl6P1nWP9lWrWlum6c050gq


In [91]:
# create a spotify object
spotify = spotipy.Spotify(auth=TEMP_TOKEN)

In [8]:
def find_id(track_title, artist_name, metadata_ids):
    """ 
    for searches with multiple results, all id were identical for the test cases I ran 
    some searches return no results, in this case the song is not on spotify
        confirmed by spot checks in the spotify music player
    some tracks give a 404 error, 
        these seem to exist in Spotify but 404 anyway
        not sure why the API does this but 
    """
    track_info = spotify.search(q='artist:' + artist_name + ' track:' + track_title, type='track')
    
    if track_info['tracks']['items'] == []:  # if track doesn't exist on Spotify
        return None
    else:
        # set the id to the first result
        track_id = track_info['tracks']['items'][0]['id']
        
        # check if there is a better match
        number_of_results = len(track_info['tracks']['items'])
        
        for i in range(number_of_results):
            alt_track_id = track_info['tracks']['items'][i]['id']
            if alt_track_id in metadata_ids:
                track_id = track_info['tracks']['items'][i]['id']
        
        return track_id

In [9]:
# TEST
find_id("You Can't Turn Me Off (In The Middle Of Turning Me On)", 'High Inergy', metadata_ids)

'7CfCUjAZFndcpAWLogl9J3'

### Add Spotify IDs to billboard songs

In [10]:
# load saved csv as df
df_billboard_songs = pd.read_csv('df_billboard_songs.csv', keep_default_na=False)

,song,artist,id
0,Misty,"""Groove"" Holmes",7DZsH0df0GuULl0FGwXMfd
1,What Now My Love,"""Groove"" Holmes",11Aldbvo6UCcVhBzv4oUdw
2,May The Bird Of Paradise Fly Up Your Nose,"""Little"" Jimmy Dickens",1WpoMGLjlEUHlWhilsOkJA
3,I Know I Know,"""Pookie"" Hudson",
4,Amish Paradise,"""Weird Al"" Yankovic",6nofbMbadUdrtZmIsBHyYE


In [11]:
df_billboard_songs.id.nunique()

17180

In [12]:
# start over at
start_over_at = 22953

# populate df_billboard_songs with ids, where available
for i, row in df_billboard_songs.iterrows():
    
    # start over at
    if i < start_over_at-1:
        continue
    
    # show status
    if i%10 == 0:
        print(i, end='  ')
    if i%100 == 0:
        print()
        
    # start over where we finished
    if df_billboard_songs['id'].iloc[i] != '':  
        continue
    # append id, NONE, or 'ERROR'
    else:
        artist = row[1]
        song = row[0]
        try:
            # unless there is an error, will append the id or None
            df_billboard_songs['id'].iloc[i] = find_id(song, artist, metadata_ids)
        except:  # any error needs to be dealt with manually
            # if there is an error, change id to 'ERROR'
            print('ERROR:  ', artist, song)
            df_billboard_songs['id'].iloc[i] = 'ERROR'
        # save every 1000 rows, if new
        if i%1000 == 0:
            df_billboard_songs.to_csv('df_billboard_songs_TEMP.csv', index=False)
        
# save final dataframe
df_billboard_songs.to_csv('df_billboard_songs.csv', index=False)

22960  22970  22980  22990  23000  
23010  23020  23030  23040  23050  23060  23070  23080  23090  

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Steve Allen and His Orchestra with The Copacabana Trio track:Cuando Calienta El Sol (When The Sun Is Hot)', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


23100  
ERROR:   Steve Allen and His Orchestra with The Copacabana Trio Cuando Calienta El Sol (When The Sun Is Hot)
23110  23120  23130  23140  23150  23160  23170  23180  23190  23200  
23210  23220  23230  23240  23250  23260  23270  23280  23290  23300  
23310  23320  23330  23340  23350  23360  23370  23380  23390  23400  
23410  23420  23430  23440  23450  23460  23470  23480  23490  23500  
23510  23520  23530  23540  23550  23560  23570  23580  23590  23600  
23610  23620  23630  

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': "artist:Talking Heads track:Life During Wartime (This Ain't No Party...This Ain't No Disco...This Ain't", 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


ERROR:   Talking Heads Life During Wartime (This Ain't No Party...This Ain't No Disco...This Ain't
23640  23650  23660  23670  23680  23690  23700  
23710  23720  23730  23740  23750  23760  23770  23780  23790  23800  
23810  23820  23830  23840  23850  23860  23870  23880  23890  23900  
23910  23920  23930  23940  23950  23960  23970  23980  23990  24000  
24010  24020  24030  24040  24050  24060  24070  24080  24090  24100  
24110  24120  24130  24140  24150  24160  24170  24180  24190  24200  
24210  24220  24230  24240  24250  

...

ERROR:   Thomas Rhett Featuring Reba McEntire, Hillary Scott, Chris Tomlin & Keith Urban Be A Light
27580  27590  27600  
27610  27620  27630  27640  27650  27660  27670  27680  27690  27700  
27710  27720  27730  27740  27750  27760  27770  27780  27790  27800  
27810  27820  27830  27840  27850  27860  27870  27880  27890  27900  
27910  27920  27930  27940  27950  27960  27970  27980  27990  28000  
28010  28020  28030  28040  28050  28060  28070  28080  28090  28100  
28110  28120  28130  28140  28150  28160  28170  28180  28190  28200  
28210  28220  28230  28240  28250  28260  28270  28280  28290  28300  
28310  28320  28330  28340  28350  28360  28370  28380  28390  28400  
28410  28420  28430  28440  28450  28460  28470  28480  

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Tyler, The Creator Featuring Brent Faiyaz & Fana Hues track:Sweet / I Thought You Wanted To Dance', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


ERROR:   Tyler, The Creator Featuring Brent Faiyaz & Fana Hues Sweet / I Thought You Wanted To Dance
28490  28500  
28510  28520  28530  28540  28550  28560  28570  28580  28590  28600  
28610  28620  28630  28640  28650  28660  28670  28680  28690  28700  
28710  28720  28730  28740  28750  28760  28770  28780  28790  28800  
28810  28820  28830  28840  28850  28860  28870  28880  28890  28900  
28910  28920  28930  28940  28950  28960  28970  28980  28990  29000  
29010  29020  29030  29040  29050  29060  29070  29080  29090  29100  
29110  29120  29130  29140  29150  29160  29170  29180  29190  29200  
29210  29220  29230  29240  29250  29260  29270  29280  29290  29300  
29310  29320  29330  29340  29350  29360  29370  29380  29390  29400  
29410  29420  29430  29440  29450  29460  29470  29480  29490  29500  
29510  29520  29530  29540  29550  29560  29570  29580  29590  29600  
29610  29620  29630  29640  29650  29660  29670  

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': "artist:will.i.am Featuring Miley Cyrus, French Montana, Wiz Khalifa & DJ Mustard track:Feelin' Myself", 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


29680  ERROR:   will.i.am Featuring Miley Cyrus, French Montana, Wiz Khalifa & DJ Mustard Feelin' Myself


In [13]:
# how many id have we added
sum(df_billboard_songs.id != "")

24019

In [17]:
# check how many id match the metadata_ids
sum(df_billboard_songs.id.isin(metadata_ids))

# which have spotify ids, but don't match metadata_ids?
# consider rechecking with metadata_ids_SQL

5694

In [33]:
# check how many id match the ids from the SQLite database
metadata_ids_SQL = pd.read_csv('all_ids_sql.csv', header=None, names=['id'])
metadata_ids_SQL.shape

(8741672, 1)

In [35]:
metadata_ids_SQL = list(metadata_ids_SQL.id)

In [37]:
sum(df_billboard_songs.id.isin(metadata_ids_SQL))

14313

In [86]:
# separate matching from non-matching dataframes
df_missing = df_billboard_songs[~df_billboard_songs.id.isin(metadata_ids_SQL)]
df_missing = df_missing[~df_missing.id.isin(metadata_ids)]  # 9650 without this, 7128 with (2500 in smaller dataset)
df_missing = df_missing[(df_missing.id!='') & (df_missing.id!='ERROR')].reset_index() # keep old index for merging later
df_missing.shape

(7128, 4)

In [87]:
df_missing.head()

,index,song,artist,id
0,1,What Now My Love,"""Groove"" Holmes",11Aldbvo6UCcVhBzv4oUdw
1,24,Music Of My Heart,'N Sync & Gloria Estefan,5smPCwVq1pp9b9mnhKRxQj
2,78,Watch Out,2 Chainz,0w2MsTyHDjPxNqqXY5o8wb
3,95,El Trago (The Drink),2 In A Room,14gC7IS8BFlzabcoqR1iNW
4,97,Walk Tall,2 Of Clubs,6byQuxXOHoos4pB8AnzuM3


In [45]:
# search again for df_missing using metadata_ids_SQL

def find_exact_id(track_title, artist_name, metadata_ids):
    """ 
    same as above, but returns 'MISSING' if not found in metadata_ids
    """
    track_info = spotify.search(q='artist:' + artist_name + ' track:' + track_title, type='track')
    
    if track_info['tracks']['items'] == []:  # if track doesn't exist on Spotify
        return 'MISSING'
    else:
        # check if there is a match
        number_of_results = len(track_info['tracks']['items'])
        
        for i in range(number_of_results):
            track_id = track_info['tracks']['items'][i]['id']
            if track_id in metadata_ids:
                return track_id
        
        # if we haven't returned a match, return 'MISSING'
        return 'MISSING'


In [88]:
# TEST
find_id("You Can't Turn Me Off (In The Middle Of Turning Me On)", 'High Inergy', metadata_ids)

'7CfCUjAZFndcpAWLogl9J3'

In [93]:
# TEST
find_exact_id("You Can't Turn Me Off (In The Middle Of Turning Me On)", 'High Inergy', metadata_ids_SQL)

'7CfCUjAZFndcpAWLogl9J3'

In [89]:
# TEST
find_exact_id("You can't find this song", 'not real artist', metadata_ids_SQL)

'MISSING'

In [94]:
# start over at
start_over_at = 0

# populate df_missing with ids, where available, 'MISSING' otherwise
for i, row in df_missing.iterrows():
        
    # start over at
    if i < start_over_at-1:
        continue
        
    # show status
    if i%10 == 0:
        print(i, end='  ')
    if i%100 == 0:
        print()
        
    # UPDATE THIS IF DROP INDEX IN RESET INDEX
    artist = row[2]
    song = row[1]
    
    try:
        # unless there is an error, will append the id or None
        df_missing['id'].iloc[i] = find_exact_id(song, artist, metadata_ids_SQL)
    except:  # any error needs to be dealt with manually
        # if there is an error, change id to 'ERROR'
        print('ERROR:  ', artist, song)
        df_missing['id'].iloc[i] = 'ERROR'
    
    # save every 1000 rows
    if i%1000 == 0:
        df_missing.to_csv('df_missing_TEMP.csv', index=False)
        
# save final dataframe
df_missing.to_csv('df_missing.csv', index=False)

0  
10  20  30  40  50  60  70  80  90  100  
110  120  130  140  150  160  170  180  190  200  
210  220  230  240  250  260  270  280  290  300  
310  320  330  340  350  360  370  380  390  400  
410  420  430  440  450  460  470  480  490  500  
510  520  530  540  550  560  570  580  590  600  
610  620  630  640  650  660  670  680  690  700  
710  720  730  740  750  760  770  780  790  800  
810  820  830  840  850  860  870  880  890  900  
910  920  930  940  950  960  970  980  990  1000  
1010  1020  1030  1040  1050  1060  1070  1080  1090  1100  
1110  1120  1130  1140  1150  1160  1170  1180  1190  1200  
1210  1220  1230  1240  1250  1260  1270  1280  1290  1300  
1310  1320  1330  1340  1350  1360  1370  1380  1390  1400  
1410  1420  1430  1440  1450  1460  1470  1480  1490  1500  
1510  1520  1530  1540  1550  1560  1570  1580  1590  1600  
1610  1620  1630  1640  1650  1660  1670  1680  1690  1700  
1710  1720  1730  1740  1750  1760  1770  1780  1790  1800  
1810  

### RE-RUN BELOW (load csv first)

In [ ]:
# RELOAD FROM CSV

df_missing = pd.read_csv('df_missing.csv', keep_default_na=False)
df_missing.head()

In [56]:
# number of tracks missing from both datasets, but on Spotify
sum(df_missing.id == 'MISSING')

# OPTIONAL: could use API to get these sound features

1265

In [65]:
df_missing.shape[0], sum(df_missing.id == 'MISSING') + sum(df_missing.id != 'MISSING')

(9650, 9650)

In [ ]:
# revised_ids = 
df_missing[(df_missing.id != 'MISSING') & (df_missing.id != '')]

In [55]:
# how many tracks are in 1.2M but not 8M?



# what about from the billboard 100?




1265

In [ ]:
# recombine df_missing with df_billboard_songs
# separate songs still missing
    # not on spotify 
    # on spotify, but not either dataset



In [ ]:
# check the percentage of songs accounted for



In [ ]:
# now merge with entire billboard list



# count the percentage of songs accounted for in the entire set

### 8+ M. Spotify Tracks, Genre, Audio Features (SQLite)
https://www.kaggle.com/datasets/maltegrosse/8-m-spotify-tracks-genre-audio-features

In [ ]:
url_8M_sql = 'D:\RYERSON\820\Datasets\8+ M. Spotify Tracks, Genre, Audio Features\spotify.sqlite'
url_8M_csv = 'all_audio_features_sql.csv'  # .gitignore (very big)


### Merge all Datasets into Billboard Features, 
export Billboard Features and All Features